# QED→Lattice **Phase Mapper v3 — Structure Hunt (Patched)**


**What this notebook does**
- One-normalization to match **α(M_Z)** exactly (within numeric tolerance)
- Scans **phase φ** and **phase offsets (δ₂, δ₃)** to find **knots/ridges** where triple-closure approaches
- Adds **torsion** between U(1) and SU(3) stiffnesses to unlock a non-degenerate helix
- Maps **loss surfaces** and exports summaries/plots

**Outputs** (written next to this notebook):
- `v3_phase_scan_summary.json`  — best φ and loss (1D scan)
- `v3_knot_d2_d3.json`          — best (δ₂, δ₃) and loss
- `v3_best_phi_r32.json`        — best (φ, r₃₂) and loss
- Plots: `v3_residuals_vs_phi.png`, `v3_lossmap_d2_d3.png`, `v3_lossmap_phi_r32.png`, braids.


In [1]:

# --- Cell 1: Imports & constants ---
import math, json, itertools
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# PDG targets
PDG_alpha_MZ    = 1/127.955
PDG_sin2W_MZ    = 0.23122
PDG_alpha_s_MZ  = 0.1179

# Scales & constants
MZ_GEV = 91.1876
HBAR   = 6.582119569e-22  # MeV·s
C      = 299792458.0      # m/s
HBARC  = 0.1973269804     # GeV·fm

# One-loop beta coefficients (MSbar, nf≈5 near MZ)
b1, b2, b3 = 41/10, -19/6, -7

# Bridge inputs (edit if desired)
omega_c = 1e23   # Hz
m_Gamma = 17.0   # MeV
chi      = 1.0   # dimensionless

# Derived bridge scales
mu_c_MeV = HBAR * omega_c
MU_C_GEV = mu_c_MeV / 1e3
xi_Gamma = (C / max(omega_c, 1e-30)) * (chi ** (-0.5))

outdir = Path(__file__).parent if "__file__" in globals() else Path.cwd()
print("Outputs will be saved in:", outdir)
print("MU_C_GEV, xi_Gamma[m]:", MU_C_GEV, xi_Gamma)


: 

In [2]:

# --- Cell 2: Exponents & stiffness dictionary ---
EXPO = {
  "U1":  dict(alpha=+0.00, beta=-1.0, gamma=+1/3, p=1.0),
  "SU2": dict(alpha=+0.00, beta=-1.0, gamma=+1/3, p=1.0),
  "SU3": dict(alpha=-0.05, beta=-1.0, gamma=+1/3, p=1.0),
}

def hat_Ki(mu_c_GeV, xi_Gamma, chi, h_i, expo):
    xi_term = (xi_Gamma if xi_Gamma != 0 else 1.0)
    return (h_i**expo["p"]) * ( (xi_term/max(1e-30, 1.0))**expo["beta"] ) * (chi**expo["gamma"]) * (max(mu_c_GeV,1e-12)**expo["alpha"])

def make_barrier_K(mu_c_GeV, xi_Gamma, chi, h_tuple, N=1.0):
    h1,h2,h3 = h_tuple
    K1 = N * hat_Ki(mu_c_GeV, xi_Gamma, chi, h1, EXPO["U1"])
    K2 = N * hat_Ki(mu_c_GeV, xi_Gamma, chi, h2, EXPO["SU2"])
    K3 = N * hat_Ki(mu_c_GeV, xi_Gamma, chi, h3, EXPO["SU3"])
    return K1,K2,K3


In [3]:

# --- Cell 3 (PATCHED): Safe one-loop runners & normalization ---
def run_alpha_to_scale(alpha_mu0, beta_coeff, mu0, mu):
    alpha_mu0 = float(alpha_mu0)
    beta_coeff = float(beta_coeff)
    mu0 = float(mu0); mu = float(mu)
    if alpha_mu0 <= 0 or mu0 <= 0 or mu <= 0:
        return float("nan")
    denom = (1.0/alpha_mu0) - (beta_coeff/(2.0*math.pi)) * math.log(mu/mu0)
    if not math.isfinite(denom) or abs(denom) < 1e-14:
        return float("nan")
    return 1.0/denom

def run_to_MZ_pair(alpha1_mu, alpha2_mu, MU0_GEV, MZ=MZ_GEV):
    a1_MZ = run_alpha_to_scale(alpha1_mu, b1, MU0_GEV, MZ)
    a2_MZ = run_alpha_to_scale(alpha2_mu, b2, MU0_GEV, MZ)
    return a1_MZ, a2_MZ

def normalize_once_to_alpha_em(K1, K2, MU0_GEV, target_alpha_em=PDG_alpha_MZ):
    K2 = abs(float(K2))  # SU(2) positive on broken sheet
    K1 = float(K1)

    def alpha_em_from_scale(s):
        a1_mu0 = (5.0/3.0) / (4*math.pi*(s*K1))
        a2_mu0 = 1.0        / (4*math.pi*(s*K2))
        A1_MZ, A2_MZ = run_to_MZ_pair(a1_mu0, a2_mu0, MU0_GEV, MZ_GEV)
        if not (math.isfinite(A1_MZ) and math.isfinite(A2_MZ) and A1_MZ>0 and A2_MZ>0):
            return float("inf")
        return (A1_MZ*A2_MZ)/(A1_MZ + A2_MZ)

    lo, hi = -36.0, +36.0
    flo = alpha_em_from_scale(10**lo)
    fhi = alpha_em_from_scale(10**hi)
    for _ in range(16):
        if flo > target_alpha_em or not math.isfinite(flo):
            lo -= 2.0; flo = alpha_em_from_scale(10**lo)
        if fhi < target_alpha_em or not math.isfinite(fhi):
            hi += 2.0; fhi = alpha_em_from_scale(10**hi)

    mid = 0.0
    for _ in range(120):
        mid = 0.5*(lo+hi)
        fmid = alpha_em_from_scale(10**mid)
        if not math.isfinite(fmid):
            hi = 0.75*hi + 0.25*lo
            continue
        if fmid < target_alpha_em:
            hi = mid
        else:
            lo = mid
    return 10**(0.5*(lo+hi))


In [4]:

# --- Cell 4: Lattice observable + full pipeline ---
C_SIGMA = None

def predict_lattice_observables(K3, xi_Gamma, c_sigma=None, K3_ref=None):
    global C_SIGMA, HBARC
    xi_fm = xi_Gamma / 1e-15
    if xi_fm <= 0: xi_fm = 1e-6
    cS = c_sigma if c_sigma is not None else C_SIGMA
    if cS is None: raise RuntimeError("C_SIGMA not set; run calibration.")
    sigma_GeV2 = cS * (K3 / max(xi_fm**2, 1e-30)) * (HBARC**2)
    if K3_ref is None: K3_ref = K3
    a0 = 0.085
    a_fm = a0 * ((K3_ref + 1e-12) / (K3 + 1e-12))**0.10
    a_fm = max(0.05, min(0.14, a_fm))
    return a_fm, sigma_GeV2

def run_full_pipeline(K1, K2, K3, MU0_GEV):
    a1_c = (5.0/3.0) / (4*math.pi*K1)
    a2_c = 1.0        / (4*math.pi*K2)
    a3_c = 1.0        / (4*math.pi*K3)
    a1_MZ = run_alpha_to_scale(a1_c, b1, MU0_GEV, MZ_GEV)
    a2_MZ = run_alpha_to_scale(a2_c, b2, MU0_GEV, MZ_GEV)
    a3_MZ = run_alpha_to_scale(a3_c, b3, MU0_GEV, MZ_GEV)
    alpha_em_MZ = (a1_MZ * a2_MZ) / (a1_MZ + a2_MZ)
    sin2W_MZ    = a1_MZ / (a1_MZ + a2_MZ)
    alpha_s_MZ  = a3_MZ
    a_fm, sigma_GeV2 = predict_lattice_observables(K3, xi_Gamma, K3_ref=K3)
    sqrt_sigma = math.sqrt(abs(sigma_GeV2))
    return {"alpha_em": alpha_em_MZ, "sin2W": sin2W_MZ, "alpha_s": alpha_s_MZ,
            "a_fm": a_fm, "sqrt_sigma": sqrt_sigma}


In [5]:

# --- Cell 5: Calibrate C_SIGMA on broken (physical) sheet ---
K1_b, K2_b, K3_b = make_barrier_K(MU_C_GEV, xi_Gamma, chi, (1.0,2.0,3.0), N=1.0)
S_star = normalize_once_to_alpha_em(K1_b, abs(K2_b), MU_C_GEV, PDG_alpha_MZ)
K1_b, K2_b, K3_b = (S_star*K1_b, S_star*abs(K2_b), S_star*K3_b)

xi_fm_b = xi_Gamma / 1e-15
sigma_GeV2_tmp = (K3_b / max(xi_fm_b**2, 1e-30)) * (HBARC**2)
target = 0.44  # GeV
C_SIGMA = (target**2) / max(sigma_GeV2_tmp, 1e-30)
print("C_SIGMA calibrated:", C_SIGMA)


C_SIGMA calibrated: 1.9359999999999997e+29


In [6]:

# --- Cell 6: Parameters (STRUCTURE HUNT) ---
r12 = 10/3       # U(1):SU(2)
r32 = 1/8        # SU(3):SU(2) softness
eps = {"U1":0.12, "SU2":0.12, "SU3":0.18}
nph = {"U1":1, "SU2":2, "SU3":3}
dph = {"U1":0.0, "SU2": math.pi/3, "SU3": 2*math.pi/3}
tau = 0.03   # torsion K1<->K3

print("r12, r32, eps, nph, dph, tau")
r12, r32, eps, nph, dph, tau


r12, r32, eps, nph, dph, tau


(3.3333333333333335,
 0.125,
 {'U1': 0.12, 'SU2': 0.12, 'SU3': 0.18},
 {'U1': 1, 'SU2': 2, 'SU3': 3},
 {'U1': 0.0, 'SU2': 1.0471975511965976, 'SU3': 2.0943951023931953},
 0.03)

In [7]:

# --- Cell 7: Base K0 (broken sheet), torsion, helical modulation ---
def base_K0(sign_SU2=+1.0):
    K1_hat, K2_hat, K3_hat = make_barrier_K(MU_C_GEV, xi_Gamma, chi, (1.0,1.0,1.0), N=1.0)
    K1_0 = K1_hat * r12
    K2_0 = K2_hat / max(r12,1e-12)
    K3_0 = (K2_0 * r32)
    K2_0 = sign_SU2 * K2_0
    S_star0 = normalize_once_to_alpha_em(K1_0, abs(K2_0), MU_C_GEV, PDG_alpha_MZ)
    return S_star0*K1_0, S_star0*abs(K2_0), S_star0*K3_0

def apply_torsion(K1,K3, tau):
    if tau==0: return K1,K3
    return (K1 + tau*K3, K3 + tau*K1)

def K_phi(K1_0, K2_0, K3_0, phi, dph_local=None):
    if dph_local is None: dph_local = dph
    K1 = K1_0 * (1 + eps["U1"] * math.sin(nph["U1"]*phi + dph_local["U1"]))
    K2 = K2_0 * (1 + eps["SU2"]* math.sin(nph["SU2"]*phi + dph_local["SU2"]))
    K3 = K3_0 * (1 + eps["SU3"]* math.sin(nph["SU3"]*phi + dph_local["SU3"]))
    K1, K3 = apply_torsion(K1, K3, tau)  # before normalization
    S = normalize_once_to_alpha_em(K1, K2, MU_C_GEV, PDG_alpha_MZ)
    return S*K1, S*K2, S*K3

K1_0, K2_0, K3_0 = base_K0(sign_SU2=+1.0)
print("Base K:", K1_0, K2_0, K3_0)


Base K: 1.1118803173272824e-53 1.000692285594554e-54 1.2508653569931926e-55


In [8]:

# --- Cell 8: Sanity check (pre-scan) ---
K1t, K2t, K3t = K1_0, K2_0, K3_0
r_test = run_full_pipeline(K1t, K2t, K3t, MU_C_GEV)
print("Sanity pipeline:", r_test)
assert all([math.isfinite(v) for v in r_test.values()]), "Non-finite in pipeline."


Sanity pipeline: {'alpha_em': -0.9306377379592896, 'sin2W': -3.3928571428571423, 'alpha_s': 0.12408503172790528, 'a_fm': 0.085, 'sqrt_sigma': 1.0242918710435599e-14}


In [9]:

# --- Cell 9: 1D φ scan ---
Nphi = 720
phis = np.linspace(0, 2*np.pi, Nphi, endpoint=False)

residuals = {"sin2W":[], "alpha_s":[], "sqrt_sigma":[], "loss": []}
tracks    = {"sin2W":[], "alpha_s":[], "sqrt_sigma":[], "alpha_em":[]}
for phi in phis:
    K1,K2,K3 = K_phi(K1_0, K2_0, K3_0, phi)
    r = run_full_pipeline(K1,K2,K3, MU_C_GEV)
    e1 = abs(r["sin2W"]-PDG_sin2W_MZ)
    e2 = abs(r["alpha_s"]-PDG_alpha_s_MZ)
    e3 = abs(r["sqrt_sigma"]-0.44)
    residuals["sin2W"].append(e1)
    residuals["alpha_s"].append(e2)
    residuals["sqrt_sigma"].append(e3)
    residuals["loss"].append(e1+e2+e3)
    tracks["sin2W"].append(r["sin2W"])
    tracks["alpha_s"].append(r["alpha_s"])
    tracks["sqrt_sigma"].append(r["sqrt_sigma"])
    tracks["alpha_em"].append(r["alpha_em"])

best_idx = int(np.argmin(residuals["loss"]))
best_phi = float(phis[best_idx])
best_loss = float(residuals["loss"][best_idx])
print("best_phi, best_loss:", best_phi, best_loss)

plt.figure()
plt.plot(phis, residuals["sin2W"], label="|Δ sin²θ_W|")
plt.plot(phis, residuals["alpha_s"], label="|Δ α_s|")
plt.plot(phis, residuals["sqrt_sigma"], label="|Δ √σ|")
plt.plot(phis, residuals["loss"], label="total loss", linewidth=2)
plt.axvline(best_phi, linestyle="--")
plt.xlabel("φ (rad)"); plt.ylabel("residual"); plt.legend(); plt.tight_layout()
plt.savefig(outdir/"v3_residuals_vs_phi.png", dpi=150); plt.close()

plt.figure(); plt.plot(phis, tracks["sin2W"]); plt.axhline(PDG_sin2W_MZ, linestyle="--")
plt.xlabel("φ (rad)"); plt.ylabel("sin²θ_W"); plt.tight_layout()
plt.savefig(outdir/"v3_track_sin2W_vs_phi.png", dpi=150); plt.close()

plt.figure(); plt.plot(phis, tracks["alpha_s"]); plt.axhline(PDG_alpha_s_MZ, linestyle="--")
plt.xlabel("φ (rad)"); plt.ylabel("α_s"); plt.tight_layout()
plt.savefig(outdir/"v3_track_alpha_s_vs_phi.png", dpi=150); plt.close()

plt.figure(); plt.plot(phis, tracks["sqrt_sigma"]); plt.axhline(0.44, linestyle="--")
plt.xlabel("φ (rad)"); plt.ylabel("√σ (GeV)"); plt.tight_layout()
plt.savefig(outdir/"v3_track_sigma_vs_phi.png", dpi=150); plt.close()

with open(outdir/"v3_phase_scan_summary.json","w") as f:
    json.dump({"best_phi":best_phi,"best_loss":best_loss}, f, indent=2)


best_phi, best_loss: 0.0 4.070262174585047


: 

In [ ]:

# --- Cell 10: 2D loss map over (δ2, δ3) ---
N = 64  # grid size
d2s = np.linspace(0, 2*np.pi, N, endpoint=False)
d3s = np.linspace(0, 2*np.pi, N, endpoint=False)
L = np.zeros((N,N), dtype=float)

for i, d2 in enumerate(d2s):
    for j, d3 in enumerate(d3s):
        phis_sub = np.linspace(0, 2*np.pi, 180, endpoint=False)
        best = 1e9
        for phi in phis_sub:
            K1,K2,K3 = K_phi(K1_0, K2_0, K3_0, phi, dph_local={"U1":dph["U1"], "SU2":d2, "SU3":d3})
            r = run_full_pipeline(K1,K2,K3, MU_C_GEV)
            loss = (abs(r["sin2W"]-PDG_sin2W_MZ)
                   +abs(r["alpha_s"]-PDG_alpha_s_MZ)
                   +abs(r["sqrt_sigma"]-0.44))
            if loss < best: best = loss
        L[i,j] = best

plt.figure(figsize=(7,6))
plt.imshow(L.T, origin="lower", extent=[0,2*math.pi,0,2*math.pi], aspect="auto")
plt.colorbar(label="min loss over φ")
plt.xlabel("δ₂ (rad)"); plt.ylabel("δ₃ (rad)")
plt.tight_layout()
plt.savefig(outdir/"v3_lossmap_d2_d3.png", dpi=150); plt.close()

min_idx = np.unravel_index(np.argmin(L), L.shape)
knot_d2 = float(d2s[min_idx[0]]); knot_d3 = float(d3s[min_idx[1]])
print("Knot at δ2, δ3 =", knot_d2, knot_d3, "with loss", float(L[min_idx]))
with open(outdir/"v3_knot_d2_d3.json","w") as f:
    json.dump({"delta2":knot_d2, "delta3":knot_d3, "min_loss":float(L[min_idx])}, f, indent=2)


In [ ]:

# --- Cell 11: 2D loss map over (φ, r32) ---
r32_grid = np.array([1/12,1/10,1/8,1/7,1/6,1/5])
Nphi = 180
phis2 = np.linspace(0, 2*np.pi, Nphi, endpoint=False)

M = np.zeros((len(r32_grid), Nphi), dtype=float)
for i, r32_val in enumerate(r32_grid):
    K1_hat, K2_hat, K3_hat = make_barrier_K(MU_C_GEV, xi_Gamma, chi, (1.0,1.0,1.0), N=1.0)
    K1_0b = K1_hat * r12
    K2_0b = K2_hat / max(r12,1e-12)
    K3_0b = (K2_0b * r32_val)
    K2_0b = +1.0 * K2_0b  # broken sheet
    S_star0b = normalize_once_to_alpha_em(K1_0b, abs(K2_0b), MU_C_GEV, PDG_alpha_MZ)
    K1_0b, K2_0b, K3_0b = (S_star0b*K1_0b, S_star0b*abs(K2_0b), S_star0b*K3_0b)
    for j, phi in enumerate(phis2):
        K1,K2,K3 = K_phi(K1_0b, K2_0b, K3_0b, phi)
        r = run_full_pipeline(K1,K2,K3, MU_C_GEV)
        M[i,j] = (abs(r["sin2W"]-PDG_sin2W_MZ)
                 +abs(r["alpha_s"]-PDG_alpha_s_MZ)
                 +abs(r["sqrt_sigma"]-0.44))

plt.figure(figsize=(9,4))
plt.imshow(M, origin="lower", aspect="auto",
           extent=[0,2*math.pi,0,len(r32_grid)],
           interpolation="nearest")
plt.yticks(np.arange(len(r32_grid))+0.5, [f"{x:.3f}" for x in r32_grid])
plt.colorbar(label="loss")
plt.xlabel("φ (rad)"); plt.ylabel("r32 = K3/K2 (pre-tilt)")
plt.tight_layout()
plt.savefig(outdir/"v3_lossmap_phi_r32.png", dpi=150); plt.close()

best_idx2 = np.unravel_index(np.argmin(M), M.shape)
best_r32 = float(r32_grid[best_idx2[0]]); best_phi2 = float(phis2[best_idx2[1]])
print("Best over (φ, r32):", best_r32, best_phi2, "with loss", float(M[best_idx2]))
with open(outdir/"v3_best_phi_r32.json","w") as f:
    json.dump({"r32":best_r32, "phi":best_phi2, "min_loss":float(M[best_idx2])}, f, indent=2)


In [ ]:

# --- Cell 12: Braid (Lissajous) views ---
Nphi = len(tracks["alpha_s"])
Δs = np.array(residuals["sin2W"])
Δa = np.array(residuals["alpha_s"])
Δσ = np.array(residuals["sqrt_sigma"])

plt.figure(figsize=(5,5))
plt.plot(Δs, Δa, lw=1)
plt.xlabel("|Δ sin²θ_W|"); plt.ylabel("|Δ α_s|"); plt.title("Braid: EW vs QCD residuals")
plt.tight_layout()
plt.savefig(outdir/"v3_braid_sin2vsaS.png", dpi=150); plt.close()

plt.figure(figsize=(5,5))
plt.plot(Δa, Δσ, lw=1)
plt.xlabel("|Δ α_s|"); plt.ylabel("|Δ √σ|"); plt.title("Braid: QCD vs Lattice residuals")
plt.tight_layout()
plt.savefig(outdir/"v3_braid_aSvssigma.png", dpi=150); plt.close()

print("Saved braid plots.")
